In [45]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

BYU_UW_root = r"G:\.shortcut-targets-by-id\1k1B8zPb3T8H7y6x0irFZnzzmfQPHMRPx\Illimited Lab Projects\Research Projects\Spiders\BYU-UW"

In [46]:
file_path = os.path.join(BYU_UW_root, 'Avg_Stress_Strain_Overture_TPU.csv')
df = pd.read_csv(file_path)
strain_columns = [col for col in df.columns if 'Strain' in col]
stress_columns = [col for col in df.columns if 'Stress' in col]

strain_df = np.array(df[strain_columns])
stress_df = np.array(df[stress_columns])

In [47]:
import pandas as pd

file_path_UW = os.path.join(BYU_UW_root, 'spec0_03062025.txt')

# Read the file and find the header row
with open(file_path_UW, 'r') as f:
    lines = f.readlines()

# Locate the header row
for i, line in enumerate(lines):
    if "Time (s)" in line:  # Adjust if needed based on actual headers
        header_row = i
        break

# Read the file starting from the header row
df_UW = pd.read_csv(file_path_UW, sep="\t", skiprows=header_row)

# Clean up column names (remove extra spaces)
df_UW.columns = [col.strip() for col in df_UW.columns]
print(df_UW.head())


   Time (s)  Primary Encoder Displacement (m)  \
0      0.00                      3.331900e-07   
1      0.05                     -2.854317e-07   
2      0.10                      3.406405e-07   
3      0.15                     -2.600998e-07   
4      0.20                      2.613664e-07   

   Secondary Encoder Displacement (m)  Strain Gauge Load Cell (N)  \
0                        1.937151e-09                    2.985444   
1                        7.301569e-09                    1.600320   
2                        1.117587e-09                    2.259341   
3                        5.289912e-09                    2.591648   
4                        1.862645e-09                    2.124902   

   Primary Current (amp)  Secondary Current (amp)  Trigger  Segment No.  
0                  0.004                      0.0     -1.0          0.0  
1                  0.000                      0.0     -1.0          0.0  
2                  0.000                      0.0     -1.0          

In [48]:
arg_T = np.argmin(np.abs(df_UW['Time (s)'] - 690.25))
L = 130/2
A = 21 * 0.078294515
fig, ax = plt.subplots()

kernel_size  = 20
kernel = np.ones(kernel_size) / kernel_size
ax.plot(np.convolve(df_UW['Primary Encoder Displacement (m)'][:arg_T]/L*1000, kernel, mode='valid'), np.convolve(df_UW['Strain Gauge Load Cell (N)'][:arg_T]/A - 0.95, kernel, mode='valid'), '*')
# ax.plot(df_UW['Time (s)'][:arg_T], df_UW['Strain Gauge Load Cell (N)'][:arg_T])
# ax.plot(df_UW['Time (s)'][:arg_T], df_UW['Primary Encoder Displacement (m)'][:arg_T])

ax.set_xlabel('Strain (-)')
ax.set_ylabel('Stress (MPa)')
plt.show()

In [49]:
# plt.figure(figsize=(10, 6))
# plt.plot(np.array(strain_df)[0], np.array(stress_df)[0], label='Average test data')
# plt.xlabel('Strain')
# plt.ylabel('Stress (Pa)')
# plt.title('Strain vs Stress')

# # Linear fit
# linear_limit = 0.05
# nearest_index = (np.abs(np.array(strain_df)[0] -linear_limit)).argmin()

# linear_fit = np.mean(np.array(stress_df)[0,:nearest_index]/np.array(strain_df)[0,nearest_index])
# plt.plot([0, linear_limit], [0, linear_limit*linear_fit], label='Linear Fit', linestyle='--')

# # Cubic fit
# cubic_fit = np.polyfit(np.array(strain_df)[0], np.array(stress_df)[0], 3)
# cubic_fit_fn = np.poly1d(cubic_fit)
# # plt.plot(np.array(strain_df)[0], cubic_fit_fn(np.array(strain_df)[0]), label='Cubic Fit', linestyle='--')

# plt.legend()

In [50]:
%matplotlib qt
from scipy.optimize import curve_fit
import numpy as np
import matplotlib.pyplot as plt

# Define custom functions for various fits
def linear_func(x, a):  # Linear function (a*x)
    return a * x

def linear_quadratic_func(x, a, b):  # Linear + Quadratic (a*x + b*x^2)
    return a * x + b * x**2

def linear_cubic_func(x, a, b):  # Linear + Cubic (a*x + b*x^3)
    return a * x + b * x**3

def linear_quadratic_cubic_func(x, a, b, c):  # Linear + Quadratic + Cubic (a*x + b*x^2 + c*x^3)
    return a * x + b * x**2 + c * x**3

def order_4_func(x, a, b, c, d):  # Linear + Quadratic + Cubic (a*x + b*x^2 + c*x^3)
    return a * x + b * x**2 + c * x**3 + d * x**4

def power_law_func(x, a, b):  # Power law (a*x^b)
    return a * x**b

def exponential_func(x, a, b):  # Exponential (a * exp(b*x))
    return a * np.exp(b * x)

def rambsburg_func(x, a, b, c):  # Rambsburg (a * (1 - exp(-b*x)) + c*x)
    return a * (1 - np.exp(-b * x)) + c * x

# Example data (replace with your actual strain and stress data)
strain = np.array(strain_df).flatten()
stress = np.array(stress_df).flatten()
stress *= 1e-6  # Convert to MPa

# Perform curve fitting for each case
linear_limit = 0.03
nearest_index = (np.abs(strain -linear_limit)).argmin()
popt_linear, _ = curve_fit(linear_func, strain[:nearest_index], stress[:nearest_index])
popt_linear_quadratic, _ = curve_fit(linear_quadratic_func, strain, stress)
popt_linear_cubic, _ = curve_fit(linear_cubic_func, strain, stress)
popt_linear_quadratic_cubic, _ = curve_fit(linear_quadratic_cubic_func, strain, stress)
popt_order_4, _ = curve_fit(order_4_func, strain, stress)
popt_power_law, _ = curve_fit(power_law_func, strain[strain > 0], stress[strain > 0])
popt_exponential, _ = curve_fit(exponential_func, strain[strain > 0], stress[strain > 0])
popt_rambsburg, _ = curve_fit(rambsburg_func, strain[strain > 0], stress[strain > 0])

# Plotting
plt.figure(figsize=(10, 6))
plt.plot(strain, stress, label='Average test data')

# Linear fit

plt.plot(strain[:nearest_index], linear_func(strain[:nearest_index], *popt_linear), label='Linear Fit', linestyle='--')

# Linear + Quadratic fit
# plt.plot(strain, linear_quadratic_func(strain, *popt_linear_quadratic), label='Linear + Quadratic Fit', linestyle='--')

# Linear + Cubic fit
# plt.plot(strain, linear_cubic_func(strain, *popt_linear_cubic), label='Linear + Cubic Fit', linestyle='--')

# Linear + Quadratic + Cubic fit
# plt.plot(strain, linear_quadratic_cubic_func(strain, *popt_linear_quadratic_cubic), label='Linear + Quadratic + Cubic Fit', linestyle='--')

# Order 4 fit
plt.plot(strain, order_4_func(strain, *popt_order_4), label='Order 4 Fit', linestyle='--')

# Power law fit
plt.plot(strain, power_law_func(strain, *popt_power_law), label='Power Law Fit', linestyle='--')

# Exponential fit
# plt.plot(strain, exponential_func(strain, *popt_exponential), label='Exponential Fit', linestyle='--')

# Rambsburg fit
plt.plot(strain, rambsburg_func(strain, *popt_rambsburg), label='Rambsburg Fit', linestyle='--')

plt.xlabel('Strain')
plt.ylabel('Stress (MPa)')
plt.title('Strain vs Stress')
plt.legend()
plt.show()
popt_linear

array([96.97026914])

In [52]:
from scipy.optimize import curve_fit
import numpy as np
import matplotlib.pyplot as plt

# Define functions as before
def linear_func(x, a):  
    return a * x

def linear_quadratic_func(x, a, b):  
    return a * x + b * x**2

def linear_cubic_func(x, a, b):  
    return a * x + b * x**3

def linear_quadratic_cubic_func(x, a, b, c):  
    return a * x + b * x**2 + c * x**3

def order_4_func(x, a, b, c, d):  
    return a * x + b * x**2 + c * x**3 + d * x**4

def power_law_func(x, a, b):  
    return a * x**b

def exponential_func(x, a, b):  
    return a * np.exp(b * x)

def rambsburg_func(x, a, b, c):  
    return a * (1 - np.exp(-b * x)) + c * x

# Example data (replace with your actual strain and stress data)
strain = np.array(strain_df).flatten()
stress = np.array(stress_df).flatten()
stress *= 1e-6  # Convert to MPa

# Perform curve fitting for each case
linear_limit = 0.03
nearest_index = (np.abs(strain -linear_limit)).argmin()
popt_linear, _ = curve_fit(linear_func, strain[:nearest_index], stress[:nearest_index])
popt_linear_quadratic, _ = curve_fit(linear_quadratic_func, strain, stress)
popt_linear_cubic, _ = curve_fit(linear_cubic_func, strain, stress)
popt_linear_quadratic_cubic, _ = curve_fit(linear_quadratic_cubic_func, strain, stress)
popt_order_4, _ = curve_fit(order_4_func, strain, stress)
popt_power_law, _ = curve_fit(power_law_func, strain[strain > 0], stress[strain > 0])
popt_exponential, _ = curve_fit(exponential_func, strain[strain > 0], stress[strain > 0])
popt_rambsburg, _ = curve_fit(rambsburg_func, strain[strain > 0], stress[strain > 0])

# Define a function to calculate R²
def r_squared(y_data, y_fit):
    ss_res = np.sum((y_data - y_fit)**2)
    ss_tot = np.sum((y_data - np.mean(y_data))**2)
    return 1 - (ss_res / ss_tot)

# Calculate R² for each fit
r2_linear = r_squared(stress[:nearest_index], linear_func(strain[:nearest_index], *popt_linear))
r2_order_4 = r_squared(stress, order_4_func(strain, *popt_order_4))
r2_power_law = r_squared(stress[strain > 0], power_law_func(strain[strain > 0], *popt_power_law))
r2_rambsburg = r_squared(stress[strain > 0], rambsburg_func(strain[strain > 0], *popt_rambsburg))

# Plotting
plt.figure(figsize=(10, 6))
plt.plot(strain, stress, label='Average test data (BYU)')

# # Linear fit
# plt.plot(strain[:nearest_index], linear_func(strain[:nearest_index], *popt_linear), 
#          label=f'Linear Fit (R²={r2_linear:.4f})', linestyle='--')

# # Order 4 fit
# plt.plot(strain, order_4_func(strain, *popt_order_4), 
#          label=f'Order 4 Fit (R²={r2_order_4:.4f})', linestyle='--')

# # Power law fit
# plt.plot(strain, power_law_func(strain, *popt_power_law), 
#          label=f'Power Law Fit (R²={r2_power_law:.4f})', linestyle='--')

# # Rambsburg fit
# plt.plot(strain, rambsburg_func(strain, *popt_rambsburg), 
#          label=f'Rambsburg Fit (R²={r2_rambsburg:.4f})', linestyle='--')
kernel_size  = 20
kernel = np.ones(kernel_size) / kernel_size
arg_T = -1
plt.plot(np.convolve(df_UW['Primary Encoder Displacement (m)'][:arg_T]/L*1000, kernel, mode='valid'), np.convolve(df_UW['Strain Gauge Load Cell (N)'][:arg_T]/A - 0.95, kernel, mode='valid'),label = 'Single test at UW')

plt.xlabel('Strain')
plt.ylabel('Stress (MPa)')
plt.title('Strain vs Stress')
plt.legend()
plt.show()


In [ ]:
from scipy.interpolate import interp1d
fig, ax = plt.subplots()
ax.plot(stress, strain, '-', label='data')
stress_mask = stress >= 0
ax.plot(stress[stress_mask], strain[stress_mask], 'o', label='masked data')

order = np.argsort(stress[stress_mask])
ax.plot(stress[stress_mask][order], strain[stress_mask][order], 'o', label='sorted data')



# stress_to_strain = interp1d(stress[stress_mask][order], strain[stress_mask][order], kind='cubic', bounds_error=False, fill_value=np.nan)

np.sum(stress[stress_mask][order][1:] <= stress[stress_mask][order][:-1])

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Sort stress and corresponding strain
order = np.argsort(stress[stress_mask])
stress_sorted = stress[stress_mask][order]
# stress_sorted *= 1e-6  # Convert to MPa
strain_sorted = strain[stress_mask][order]

# Find unique stress values and compute mean strain for duplicates
unique_stress, indices, counts = np.unique(stress_sorted, return_inverse=True, return_counts=True)
unique_strain = np.zeros_like(unique_stress, dtype=float)

for i in range(len(unique_stress)):
    unique_strain[i] = np.mean(strain_sorted[indices == i])  # Average strains for duplicate stresses


stress_to_strain = interp1d(unique_stress, unique_strain, kind='cubic', bounds_error=False, fill_value=np.nan)


# Plot results
fig, ax = plt.subplots()
# ax.plot(stress, strain, '-', label='data')  # Original data
ax.plot(unique_stress, unique_strain, '.', label='merged data')  # Unique stress-strain pairs
ax.plot(np.linspace(0, 7.5, 100), stress_to_strain(np.linspace(0, 7.5, 100)), '--', label='cubic interpolation')  # Interpolated data
plt.xlabel('Stress (MPa)')
plt.ylabel('Strain')
ax.legend()
plt.show()